In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pip install --upgrade mplfinance

In [ ]:
pip install bs4

In [ ]:
pip install yfinance

In [ ]:
pip install yfinance -U

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
#import mplfinance as mpf
import pandas_datareader.data as pdr
import matplotlib.dates as mdates
import bs4 as bs
import pickle
import requests
import datetime as dt
#import fix_yahoo_finance as yf

In [ ]:
"""
style.use('ggplot')
start=dt.datetime(2000,1,1)
end=dt.datetime(2020,12,31)
df=web.DataReader('TSLA','yahoo',start,end)
print(df.head())
df.plot()
plt.show()
#df['100ma']=df['Adj Close'].rolling(window=100,min_periods=0).mean()
#df.dropna(inplace=True)
#print(df.head())
"""

In [ ]:
def save_sp500_tickers():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,'lxml')
    table=soup.find('table',{'class':"wikitable sortable"})
    #print(table)
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[0].text
        tickers.append(ticker)
    with open("sp500tickers.pickle",'wb') as f:
        pickle.dump(tickers,f)
        #print(tickers)
        str1 = " "
        label=str1.join(tickers)
        label=label.replace('\n',"")
        tickers= list(label.split(" "))
        return tickers    

#save_sp500_tickers()

In [ ]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers=save_sp500_tickers()
    else:
        with open('sp500tickers.pickle','rb') as f:
            tickers = pickle.load(f)
            
    if not os.path.exists('stock_dfs'):
        os.makedirs("stock_dfs")
        
    start = dt.datetime(2000,1,1)
    end= dt.datetime(2020,12,31)
    failed = []
    passed = []
    data = pd.DataFrame()
    for ticker in tickers:
        #print(ticker)
         if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.get_data_yahoo(ticker, start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
         else:
            print('Alredy have{}'.format(ticker))
            
get_data_from_yahoo()        
            
            

In [ ]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers=save_sp500_tickers()
    else:
        with open('sp500tickers.pickle','rb') as f:
            tickers = pickle.load(f)
            
    if not os.path.exists('stock_dfs'):
        os.makedirs("stock_dfs")
        
    start = dt.datetime(2000,1,1)
    end= dt.datetime(2020,12,31)
    failed = []
    passed = []
    data = pd.DataFrame()
    for ticker in tickers[:10]:
        try:
            data[ticker] = web.DataReader(ticker, data_source= "yahoo", start = "2019-1-1")["Adj Close"]
            passed.append(x)
        except (IOError, KeyError):
            msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
            failed.append(ticker)
        else:
            print('Alredy have{}'.format(ticker))
            
get_data_from_yahoo()        

In [ ]:
get_data_from_yahoo()

In [ ]:
failed=[]
passed=[]

def collect_data(tickers):
  mydata = pd.DataFrame()
  for t in tickers:
    try:
      mydata[t] = wb.DataReader(t,data_source='yahoo',start='01-10-2019')['Adj Close']
      passed.append(t)
    except (IOError, KeyError):
      msg= 'NaN'
      failed.append(t)

  print(mydata)
  return mydata

collect_data(tickers)

In [ ]:
start = dt(2016, 12, 31)
end = dt.now()
INPX = data.DataReader('INPX', 'yahoo', start, end)

In [ ]:
fin_prod_data = pdr.get_data_yahoo(symbol.upper(), start_date, end_date)

In [ ]:
import sys
import pandas as pd
import datetime
import numpy as np
from pandas_datareader import data, wb
import pandas_datareader as pdr
from collections import defaultdict
 
def Print_Market_Profile(symbol, height_precision = 1, 
    frequency='m', start_date=None, end_date=None):

    # We will look at stock prices over the past year
    if start_date == None:
        # get a year's worth of data from today
        start_date = datetime.date.today() - datetime.timedelta(days=365.24)
        # set date to first of month
        start_date = start_date.replace(day=1)
    if end_date == None:
        end_date = datetime.date.today() 

    fin_prod_data = pdr.get_data_google(symbol.upper(), start_date, end_date)
    fin_prod_data[('High')] = fin_prod_data[('High')] * height_precision
    fin_prod_data[('Low')] = fin_prod_data[('Low')] * height_precision
    fin_prod_data = fin_prod_data.round({'Low': 0, 'High': 0})  
     
    time_groups = fin_prod_data.groupby(pd.TimeGrouper(freq=frequency))['Adj Close'].mean()
    current_time_group_index=0
       
    from collections import defaultdict
    mp = defaultdict(str)
    char_mark = 64

    # build dictionary with all needed prices
    tot_min_price=min(np.array(fin_prod_data['Low']))
    tot_max_price=max(np.array(fin_prod_data['High']))
    for price in range(int(tot_min_price), int(tot_max_price)):
        mp[price]+=('\t')

    # add max price as it will be ignored in for range loop above
    mp[tot_max_price] = '\t' + str(time_groups.index[current_time_group_index])[5:7] + '/' + str(time_groups.index[current_time_group_index])[3:4]
             
    for x in range(0, len(fin_prod_data)):
        if fin_prod_data.index[x] > time_groups.index[current_time_group_index]:
            # new time period
            char_mark=64
            # buffer and tab all entries
            buffer_max = max([len(v) for k,v in mp.iteritems()])
            current_time_group_index += 1
            for k,v in mp.iteritems():
                mp[k] += (chr(32) * (buffer_max - len(mp[k]))) + '\t'
            mp[tot_max_price] += str(time_groups.index[current_time_group_index])[5:7] + '/' + str(time_groups.index[current_time_group_index])[3:4]
            

        char_mark += 1
        min_price=fin_prod_data['Low'][x]
        max_price=fin_prod_data['High'][x]
        for price in range(int(min_price), int(max_price)):
            mp[price]+=(chr(char_mark))
 
    sorted_keys = sorted(mp.keys(), reverse=True)
    for x in sorted_keys:
        # buffer each list
        print(str("{0:.2f}".format((x * 1.0) / height_precision)) + ': \t' + ''.join(mp[x]))
 
def main():
    # customize ingestion of agruments to handle
    # frequency: http://nullege.com/codes/search/pandas.TimeGrouper

    if (len(sys.argv[1:]) == 1):
        symbol = sys.argv[1:][0]
        Print_Market_Profile(symbol)
    elif (len(sys.argv[1:]) == 2):
        symbol = sys.argv[1:][0]
        height_precision = sys.argv[1:][1]
        Print_Market_Profile(symbol, height_precision)
    elif (len(sys.argv[1:]) == 3):
        symbol = sys.argv[1:][0]
        height_precision = sys.argv[1:][1]
        frequency = sys.argv[1:][2]
        Print_Market_Profile(symbol, height_precision, frequency)

if __name__ == "__main__":
    main()

In [ ]:
git clone https://github.com/pydata/pandas-datareader